In [363]:
! nvidia-smi

Sun Apr 10 09:51:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    31W /  70W |   4720MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [364]:
#content to google drive
from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [365]:
!pip install tensorflow==2.2.0
!pip install keras==2.4.2
!pip install segmentation_models

In [366]:
# Enter modules
import zipfile  # For faster extraction
import keras
import tensorflow as tf
import os
from numpy import expand_dims
import cv2 # for read image
import numpy as np # For data manipulation
import matplotlib.pyplot as plt # for draw
from google.colab.patches import cv2_imshow #coordinate cv2 with colab
import nibabel as nib  #To read nii format
import re  # For parsing the filenames (to know their modality)
import glob # For populating the list of files
import imgaug.augmenters as iaa #for data agumention
from sklearn.model_selection import KFold, StratifiedKFold #Data setup and data segmentation
import imgaug as ia
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import time
import datetime
import os
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K
import numpy as np
import tensorflow as tf
import segmentation_models as sm

In [367]:
import os
import nibabel as nib
Fusion=os.listdir('/content/drive/MyDrive/prostate_crop/weighted_Fusion_ADC+T2W_crop')
print(len(Fusion))

Fusion_imag=[]
X=[]
for i in Fusion:
  Fusion_imag.append('/content/drive/MyDrive/prostate_crop/weighted_Fusion_ADC+T2W_crop/' + i )
for j in Fusion_imag:
  img= nib.load(j)
  fusion_img = img.get_fdata(caching='unchanged') 
  X.append(fusion_img)

42


In [368]:
label=os.listdir('/content/drive/MyDrive/prostate_crop/Label_crop')
print(len(label))
label_imag=[]
Y=[]
for i in label:
  label_imag.append('/content/drive/MyDrive/prostate_crop/Label_crop/' + i )
for j in label_imag:
  img= nib.load(j)
  lbl_img = img.get_fdata(caching='unchanged') 
  Y.append(lbl_img)  

42


In [369]:
print(len(X),len(Y))

42 42


In [370]:
X=np.array(X)
Y=np.array(Y)

In [371]:
np.unique(Y[0])

array([0., 1., 2., 3., 4.])

In [372]:
from sklearn.model_selection import KFold
i=0 
n_split=10
 
for train_index,test_index in KFold(n_split).split(X):
  if i==9:
    X_train,X_val=X[train_index],X[test_index]
    y_train,y_val=Y[train_index],Y[test_index]
  i=i+1  


print(len(X_train),len(y_train))
print(len(X_val),len(y_val))

38 38
4 4


In [373]:
print(X_train.shape , y_train.shape ,X_val.shape, y_val.shape )

(38, 160, 160, 19) (38, 160, 160, 19) (4, 160, 160, 19) (4, 160, 160, 19)


In [374]:
X_T=[]
for i in range (0,len(X_train)):
  for j in range(0,19):
    img=X_train[i][:,:,j]
    X_T.append(img)

Y_T=[]
for i in range (0,len(y_train)):
  for j in range(0,19):
    img=y_train[i][:,:,j]
    Y_T.append(img)    

X_V=[]
for i in range (0,len(X_val)):
  for j in range(0,19):
    img=X_val[i][:,:,j]
    X_V.append(img) 
Y_V=[]
for i in range (0,len(y_val)):
  for j in range(0,19):
    img=y_val[i][:,:,j]
    Y_V.append(img)              

In [375]:
np.unique(Y_V[0])

array([0., 3.])

In [376]:
print(len(X_T) , len(Y_T), len(X_V),len(Y_V))

722 722 76 76


In [377]:
Y_T_c=to_categorical(Y_T,5)
Y_V_c=to_categorical(Y_V,5)

In [378]:
np.unique(Y_T_c[0])

array([0., 1.], dtype=float32)

In [379]:
X_T=np.array(X_T)
Y_T_c=np.array(Y_T_c)
X_V=np.array(X_V)
Y_V_c=np.array(Y_V_c)

In [380]:
X_T=np.expand_dims(X_T, axis=-1)
X_V=np.expand_dims(X_V, axis=-1)

In [381]:
print(X_T.shape , Y_T_c.shape ,X_V.shape, Y_V_c.shape )

(722, 160, 160, 1) (722, 160, 160, 5) (76, 160, 160, 1) (76, 160, 160, 5)


In [382]:
import random
def Data_generator_train(X,y, batch_size=8):   
    while True:               
          for start in range(0, X.shape[0], batch_size):
              x_batch = []
              y_batch = []
              end = min(start + batch_size, X.shape[0])
              ids_batch = X[start:end]
              for idd in range(start,end):           
                   a=random.randint(0,2)
                   if a==0:
                      image =np.fliplr(X[idd]) 
                      label =np.fliplr(y[idd])
                   if a==1:
                     image =np.flipud(X[idd]) 
                     label =np.flipud(y[idd])
                   elif a==2:
                     image =(X[idd]) 
                     label =(y[idd])                
                   x_batch.append(image)
                   y_batch.append(label)               
              x_batch= np.array(x_batch)
              y_batch= np.array(y_batch)                                           
              yield x_batch, y_batch


In [383]:
m,b=next(iter(Data_generator_train(X_T,Y_T_c)))

print(m.shape)
print(b.shape)

(8, 160, 160, 1)
(8, 160, 160, 5)


In [384]:
import random
def Data_generator_test(X,y, batch_size=1):   
    while True:               
          for start in range(0, X.shape[0], batch_size):
              x_batch = []
              y_batch = []
              end = min(start + batch_size, X.shape[0])
              ids_batch = X[start:end]
              for idd in range(start,end):           
                image =(X[idd]) 
                label =(y[idd])                
                x_batch.append(image)
                y_batch.append(label)               
              x_batch= np.array(x_batch)
              y_batch= np.array(y_batch)                                           
              yield x_batch, y_batch


In [385]:
m,b=next(iter(Data_generator_test(X_V,Y_V_c)))

print(m.shape)
print(b.shape)

(1, 160, 160, 1)
(1, 160, 160, 5)


In [386]:
model= sm.Unet('resnet34', classes=5, activation='softmax', encoder_weights=None, input_shape=(160, 160, 1))

In [387]:
optim = tf.keras.optimizers.Adam(lr =0.0008)
dice_loss = sm.losses.DiceLoss() 
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5), 'accuracy']

In [388]:
model.compile(optim, total_loss, metrics)

In [389]:
def step_decay(epoch):
     
     initAlpha = 0.0008
     factor = .9
     dropEvery = 30
     alpha = initAlpha * (factor ** np.floor((1 + epoch) / dropEvery))
     return float(alpha)

In [390]:
batch_size =8
TrainSteps = len(X_T)  / batch_size
ValSteps   = len(X_V)

In [391]:
model.load_weights('/content/drive/My Drive/weighted_Fusion_crop/unet_resnet34_weighted_Fusion_ADC+T2W_crop/unet_resnet34_fold10.h5')

In [392]:
scores=model.evaluate(X_V,Y_V_c , batch_size=1)
print("Loss: {:.5}".format(scores[0]))
for metric, value in zip(metrics, scores[1:]):
    print("mean {}: {:.5}".format(metric, value))

76/76 [==============================] - 1s 10ms/step - loss: 1.0408 - iou_score: 0.3777 - f1-score: 0.4089 - accuracy: 0.7057
Loss: 1.0408
mean <segmentation_models.metrics.IOUScore object at 0x7f1cc0fef350>: 0.37774
mean <segmentation_models.metrics.FScore object at 0x7f1cc0fef990>: 0.40892
mean accuracy: 0.70567


In [393]:
p=model.predict(X_V)
y=Y_V_c

In [394]:

from scipy.spatial.distance import directed_hausdorff


def Average(lst):
    return sum(lst) / len(lst)



HD=[]
for i in range(0,76):
  hd=directed_hausdorff(p[i,:,:,0], y[i,:,:,0])
  HD.append(hd[0])

print(len(HD))
print(Average(HD))

76
4.326417553812298


/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:456: UserWarning: you are shuffling a '_memoryviewslice' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  result = _hausdorff.directed_hausdorff(u, v, seed)


In [395]:

from scipy.spatial.distance import directed_hausdorff


def Average(lst):
    return sum(lst) / len(lst)



HD=[]
for i in range(0,76):
  hd=directed_hausdorff(p[i,:,:,1], y[i,:,:,1])
  HD.append(hd[0])

print(len(HD))
print(Average(HD))

/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:456: UserWarning: you are shuffling a '_memoryviewslice' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  result = _hausdorff.directed_hausdorff(u, v, seed)


76
0.0002466353571239049


In [396]:

from scipy.spatial.distance import directed_hausdorff


def Average(lst):
    return sum(lst) / len(lst)



HD=[]
for i in range(0,76):
  hd=directed_hausdorff(p[i,:,:,2], y[i,:,:,2])
  HD.append(hd[0])

print(len(HD))
print(Average(HD))

/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:456: UserWarning: you are shuffling a '_memoryviewslice' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  result = _hausdorff.directed_hausdorff(u, v, seed)


76
0.0004732217633637819


In [397]:

from scipy.spatial.distance import directed_hausdorff


def Average(lst):
    return sum(lst) / len(lst)



HD=[]
for i in range(0,76):
  hd=directed_hausdorff(p[i,:,:,3], y[i,:,:,3])
  HD.append(hd[0])

print(len(HD))
print(Average(HD))

76
4.348438642892321


/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:456: UserWarning: you are shuffling a '_memoryviewslice' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  result = _hausdorff.directed_hausdorff(u, v, seed)


In [398]:

from scipy.spatial.distance import directed_hausdorff


def Average(lst):
    return sum(lst) / len(lst)



HD=[]
for i in range(0,76):
  hd=directed_hausdorff(p[i,:,:,4], y[i,:,:,4])
  HD.append(hd[0])

print(len(HD))
print(Average(HD))

/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:456: UserWarning: you are shuffling a '_memoryviewslice' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  result = _hausdorff.directed_hausdorff(u, v, seed)


76
1.8900847197817185e-05
